<h2>FT003</h2>
<p> En este apartado se trata la base de datos FT003 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [ ]:
# Se carga el FT003 para 2020 y 2021 completo desde el archivo .dta
FT003_2021_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT003 20202021.dta')
# Se carga el FT003 para diciembre de 2019 desde el archivo .dta
FT003_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT003 122019.dta')
# Se concatenan la bases de datos 
frames = [FT003_2021_2021, FT003_2019]
FT003 = pd.concat(frames, ignore_index=True)
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT003_2021_2021, FT003_2019, frames]

In [2]:
# Se carga el FT005 completo desde el archivo .dta
FT003 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/FT003_1219_0521.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object, 'Fecha': object})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003.rename(columns={'Nit': 'nit_eps', 'Año': 'ano', 'Periodo': 'mes', 'lineaNegocio': 'linea','tipoIdDeudor': 'tipoiddeudor', 'idDeudor': 'nitproveedor', 'conceptoDeudores': 'concepto', 'tipoDeuda': 'tipodeuda', 'medicionPosterior': 'medicion', 'cxcPendientesRadicar': 'cxcsinradicar','cxcNoVencidas': 'cxcnovencidas', 'cxcMora30Dias': 'cxc30', 'cxcMora60Dias': 'cxc60', 'cxcMora90Dias': 'cxc90','cxcMora180Dias': 'cxc180', 'cxcMora360Dias': 'cxc360', 'cxcMoraMayor360Dias': 'cxcmayor360'}, inplace=True)

In [4]:
# Se transforman los datos para su correcta identificación
FT003[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoiddeudor',
       'nitproveedor', 'dvDeudor', 'nombreDeudor', 'codigoMunicipio',
       'concepto', 'tipodeuda', 'medicion']] = FT003[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea', 'tipoiddeudor',
       'nitproveedor', 'dvDeudor', 'nombreDeudor', 'codigoMunicipio',
       'concepto', 'tipodeuda', 'medicion']].astype(str)

FT003[['cxcsinradicar', 'cxcnovencidas',
       'valorMercado', 'cxc30', 'cxc60', 'cxc90', 'cxc180', 'cxc360',
       'cxcmayor360', 'deterioro30Dias', 'deterioro60Dias', 'deterioro90Dias',
       'deterioro180Dias', 'deterioro360Dias', 'deterioroMayor360Dias',
       'ajuste', 'saldo']] = FT003[['cxcsinradicar', 'cxcnovencidas',
       'valorMercado', 'cxc30', 'cxc60', 'cxc90', 'cxc180', 'cxc360',
       'cxcmayor360', 'deterioro30Dias', 'deterioro60Dias', 'deterioro90Dias',
       'deterioro180Dias', 'deterioro360Dias', 'deterioroMayor360Dias',
       'ajuste', 'saldo']].astype(float)

In [5]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT003 = FT003.loc[FT003['linea'] == '1']

In [6]:
# Se crea la columna Cuentas por Cobrar (CXC) sumando las columnas cxcsinradicar, cxcnovencidas, cxc30, cxc60, cxc90, cxc180, cxc360 y cxcmayor360
CXC = FT003['cxcsinradicar'] + FT003['cxcnovencidas'] + FT003['cxc30'] + FT003['cxc60'] + FT003['cxc90'] + FT003['cxc180'] + FT003['cxc360'] + FT003['cxcmayor360']
FT003['CXC'] = CXC

In [7]:
# Para hacer la suma de CXC por NIT EPS, tipo de proveedor, NIT proveedor, Mes y Año
FT003 = FT003.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXC': 'sum'}).reset_index()

In [8]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de deudor, NIT proveedor, Mes y Año. Para esto se cuenta el número de duplicados
duplicadosFT003 = FT003[FT003.duplicated(['nit_eps', 'nitproveedor', 'mes', 'ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT003.nit_eps.count(), sep='\n')

El número de registros duplicados es:
0


In [9]:
# Se genera una nueva columna extrayendo el primer caracter del ID nitproveedor
FT003['d1'] = FT003['nitproveedor'].str[:1]

In [10]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT003['largo'] = FT003.nitproveedor.str.len()

In [11]:
# Se cuenta el número de duplicados para hacer la respectiva corrección del tipo id deudor
df_tipoiddeudor = FT003.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).size().reset_index().rename(columns={0:'d'})
# df_tipoiddeudor.sort_values(by='d', ascending=False)

In [12]:
# Se crea la unión de del FT003 con la cuenta de repetidos según NIT EPS, NIT proveedor, mes y año
FT003 = FT003.merge(df_tipoiddeudor, left_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], how='left')

In [13]:
# Para hacer la suma de CXC por NIT EPS, tipo de deudor corregido, NIT proveedor, Mes y Año
FT003 = FT003.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXC': 'sum'}).reset_index()

In [14]:
# Se convierten las columnas de mes y año en int para mejorar su manipulación
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(int)

In [15]:
# Se libera memoria
del [CXC, df_tipoiddeudor, duplicadosFT003]

In [16]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT003 = FT003.set_index('mes').groupby(['nit_eps', 'nitproveedor', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'nitproveedor', 'ano'], 1).reset_index()

In [17]:

# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT003[
    (FT003['ano'] == 2019) & (FT003['mes'] == 1) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 2) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 3) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 4) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 5) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 6) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 7) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 8) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 9) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 10) |
    (FT003['ano'] == 2019) & (FT003['mes'] == 11)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT003 = FT003.drop(df_filtered)

In [18]:
# Se completan los años y los meses para las observaciones que no presentaron ninguna observación en un año de interés. Por ejemplo, a aquellos sujetos que reportaron cifras en 2019, pero no en 2020, se les completa el año 2020 para poder calcular la diferencia entre diciembre de 2019 y enero de 2020
# Se convierte la variable en mes y año
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(float)
FT003['periodo'] = pd.to_datetime(FT003.ano*10000 + FT003.mes*100 + 1, format='%Y%m%d')
fecha_ideal = pd.date_range('12-01-2019','05-01-2021',freq='M')

In [19]:
# Se agrupa la información según NIT EPS, NIT proveedor y periodo sumando el valor de CXC que corresponda a cada agrupación descrita
FT003_dif = FT003.groupby(['nit_eps', 'nitproveedor', pd.Grouper(key='periodo', freq='M')])[['CXC']].sum().reset_index()

In [20]:
# Se prepara el dataframe para calcular la diferencia periodo a periodo (t+1 - t) respetando la agregación anterior
FT003_dif = FT003_dif.set_index('periodo').groupby(['nit_eps', 'nitproveedor']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'nitproveedor'], 1).reset_index()

In [21]:
FT003_dif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1899036 entries, 0 to 1899035
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   nit_eps       object        
 1   nitproveedor  object        
 2   level_2       datetime64[ns]
 3   CXC           float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 58.0+ MB


In [22]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003_dif.rename(columns={'level_2': 'periodo'}, inplace=True)

In [23]:
# Se establece el index (identificación) como NIT EPS, NIT proveedor y el periodo para cada observación
FT003_dif = FT003_dif.set_index(['nit_eps', 'nitproveedor', 'periodo'])

In [24]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas (t+1 - t) y se llenan los espacios vacíos con ceros
FT003_dif = FT003_dif.diff().fillna(0).reset_index()

In [25]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT003_dif.rename(columns={'CXC': 'DCXCmes'}, inplace=True)

#  Se extrae el mes y el año para porterior left join con el agregado
FT003_dif['ano'] = FT003_dif['periodo'].dt.year
FT003_dif['mes'] = FT003_dif['periodo'].dt.month

#Se transforman las variables a STR para mejorar su manipulación
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(int)

# Se eliminan las columnas que no son de interés
FT003 = FT003.drop(['periodo'], axis=1)
FT003_dif = FT003_dif.drop(['periodo'], axis=1)

In [26]:
# Se unen las dos bases de datos para obtener la información de diferenciada mensualmente y agregada de las CXC en una sola base de datos. Se sobreescribe el FT003
FT003 = pd.merge(FT003_dif, FT003, how='left', left_on=['nit_eps', 'nitproveedor', 'mes', 'ano'], right_on=['nit_eps', 'nitproveedor', 'mes', 'ano'])

In [27]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT003[
    (FT003['ano'] == 2019)
    ].index

# Se eliminan las observaciones que cumplen las condiciones anteriore
FT003 = FT003.drop(df_filtered)

In [29]:
# Se convierte la variable en mes y año
FT003[['ano', 'mes']] = FT003[['ano', 'mes']].astype(float).astype(int)

In [31]:
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT003 = FT003.groupby(['nit_eps', 'nitproveedor', 'mes', 'ano']).agg({'CXC':'sum', 'DCXCmes':'sum'}).reset_index()

In [32]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT003_pres_ope.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cxcprestador_2020_2021.xlsx', index = False)
FT003.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/cxcprestador_2020_ene_may_2021.csv', index = False, encoding='utf-8-sig')

In [36]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT003, FT003_dif, df_filtered, fecha_ideal]